In [1]:
from openai import AzureOpenAI
from openai import OpenAI

In [2]:
# api_version = "2024-07-18"
api_version = "2024-02-15-preview"
azure_endpoint="https://nshan-m3oer8jc-eastus.openai.azure.com/"
azureClient = AzureOpenAI(api_version=api_version, azure_endpoint=azure_endpoint)
# azureClient = AzureOpenAI(azure_endpoint=azure_endpoint)

In [3]:
q = 'the course has already started, can I still enroll?'
response = azureClient.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

In [4]:
response.choices[0].message.content

"It's possible to enroll in a course after it has started, but this generally depends on the institution, the specific course, and its policies. Here are a few steps you can take to find out:\n\n1. **Check the Course Details:** Look at the course’s official page or syllabus to see if there’s any information about enrollment deadlines.\n\n2. **Contact the Instructor:** Reach out directly to the course instructor or professor. They may be able to provide exceptions or additional guidance.\n\n3. **Contact the Admissions Office:** The admissions or registrar's office can usually give you information about the possibility of late enrollment and any potential penalties or fees.\n\n4. **Consider Auditing:** Some institutions allow students to audit courses, which means you can attend and participate without receiving credit. This might be a good option if formal enrollment is no longer possible.\n\n5. **Look for Online Options:** If physical attendance isn't possible, there might be an online

In [11]:
from langchain.document_loaders import PyPDFLoader

# Load the book
loader = PyPDFLoader("https://services.google.com/fh/files/misc/practitioners_guide_to_mlops_whitepaper.pdf")
pages = loader.load_and_split()

### Pre processing

Remove non printable char & extra spaces

In [12]:
# Cut out the open and closing parts
pages = pages[6:1308]
# Combine the pages, and replace the tabs with spaces
text = ' '.join([page.page_content.replace('\t', ' ') for page in pages])

In [13]:
text[0:200]

'7\nThe MLOps lifecycle\nThe MLOps lifecycle encompasses seven integrated and iterative processes, as shown in figure 2.\nThe processes can consist of the following:\n• ML development concerns experimentin'

In [20]:
import re
def clean_text(text):
   # Remove extra spaces
   cleaned_text = re.sub(r' +', ' ', text)
   # Remove non-printable characters, optionally preceded by 'David Copperfield'
   cleaned_text = re.sub(r'[\x00-\x1F]', '', cleaned_text)
   # Replace newline characters with spaces
   cleaned_text = cleaned_text.replace('\n', ' ')
   # Remove spaces around hyphens
   cleaned_text = re.sub(r'\s*-\s*', '', cleaned_text)
   return cleaned_text

In [21]:
clean_text=clean_text(text)

In [26]:
from langchain_openai import AzureOpenAI

In [31]:
llm = AzureOpenAI(
    model='gpt-4o'
)

In [32]:
llm.get_num_tokens(clean_text)

9283

Split doc using SemanticChunker

In [35]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
text_splitter = SemanticChunker(
   OpenAIEmbeddings(), breakpoint_threshold_type="interquartile"
)
docs = text_splitter.create_documents([clean_text])

In [37]:
import numpy as np
import openai
def get_embeddings(text):
   response = AzureOpenAI.embeddings.create(
       model="text-embedding-3-small",
       input=text
   )
   return response.data
embeddings=get_embeddings([doc.page_content for doc in docs]
)

AttributeError: embeddings